In [1]:
import minedojo
# import matplotlib.pyplot as plt
from PIL import Image
import time
import cv2
import glob
import numpy as np

import hashlib
import hydra
import torch
from hydra import compose, initialize
from omegaconf import OmegaConf
from mineclip import MineCLIP

[INFO:minedojo.tasks] Loaded 1581 Programmatic tasks, 1560 Creative tasks, and 1 special task: "Playthrough". Totally 3142 tasks loaded.
/mnt/e/AutonoMC/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def main(cfg):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    path = cfg.pop('ckpt_path', None)

    # print('cfg', cfg)
    # print('device', device)
    model = MineCLIP(**cfg).to(device)
    model.load_ckpt(path, strict=True)
    print("Successfully loaded ckpt")

    return model


cfg = {
    'arch': 'vit_base_p16_fz.v2.t2',
    'hidden_dim': 512,
    'image_feature_dim': 512,
    'mlp_adapter_spec': 'v0-2.t0',
    'pool_type': 'attn.d2.nh8.glusw',
    'resolution': [160, 256],
    'ckpt_path': 'weights/attn.pth'
}

model = main(cfg)

Successfully loaded ckpt


In [3]:
frameSize = (160, 256)
env = minedojo.make(task_id='harvest_wool_with_shears_and_sheep', image_size=frameSize)
out = cv2.VideoWriter('outputs/ahhhh.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (frameSize[1], frameSize[0]))
prompts = [env.task_prompt]
images = []

obs = env.reset()
for i in range(50):
    act = env.action_space.no_op()
    act[0] = 1    # forward/backward
    if i % 10 == 0:
        act[2] = 1    # jump
    obs, reward, done, info = env.step(act)
    # print('next frame')
    # env.render()
    
    pic = obs['rgb']

    pic = pic.transpose((1, 2, 0))
    pic = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)
    
    img = pic.transpose((2, 0, 1))
    img = img[None, :, :, :]
    img = torch.Tensor(img).to('cuda')
    images.append(img)

    out.write(pic)

env.close()
out.release()

video = torch.stack(images, dim=0)
image_feats = model.forward_image_features(video)
video_feats = model.forward_video_features(image_feats)
video_feats_2 = model.encode_video(video)

text_feats_batch = model.encode_text(prompts)

VIDEO_BATCH, TEXT_BATCH = video.size(0), len(prompts)

assert text_feats_batch.shape == (TEXT_BATCH, 512)

# compute reward from features
logits_per_video, logits_per_text = model.forward_reward_head(
    video_feats, text_tokens=text_feats_batch
)
assert logits_per_video.shape == (VIDEO_BATCH, TEXT_BATCH)
assert logits_per_text.shape == (TEXT_BATCH, VIDEO_BATCH)
# directly pass in strings. This invokes the tokenizer under the hood
reward_scores_2, _ = model.forward_reward_head(video_feats, text_tokens=prompts)
# pass in cached, encoded text features
reward_scores_3, _ = model(
    video_feats, text_tokens=text_feats_batch, is_video_features=True
)
reward_scores_4, _ = model(
    video, text_tokens=text_feats_batch, is_video_features=False
)
# all above are equivalent, just starting from features or raw values

print(reward_scores_2.shape)
print(reward_scores_3.shape)
print(reward_scores_4.shape)



/mnt/e/AutonoMC/venv/lib/python3.10/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')
/mnt/e/AutonoMC/venv/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
[INFO:minedojo.tasks] Loaded 1581 Programmatic tasks, 1560 Creative tasks, and 1 special task: "Playthrough". Totally 3142 tasks loaded.


torch.Size([50, 1])
torch.Size([50, 1])
torch.Size([50, 1])


In [4]:
env.close()
cv2.destroyAllWindows()


In [5]:
env.close()
cv2.destroyAllWindows()


In [6]:
from IPython import Video



ImportError: cannot import name 'Video' from 'IPython' (/mnt/e/AutonoMC/venv/lib/python3.10/site-packages/IPython/__init__.py)

In [ ]:
cap = cv2.VideoCapture('temp/SampleVideo_1280x720_1mb.mp4')
print(cap.get(3), cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter('outputs/helpme.mp4',fourcc, 20.0,(int(cap.get(3)),int(cap.get(4))))

not_done = True
while not_done:
    not_done, frame = cap.read()
    if frame is not None:
        print(frame.shape, type(frame))
        out.write(frame)

out.release()

1280.0 720.0
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 3) <class 'numpy.ndarray'>
(720, 1280, 